In [1]:
# 대체적으로 성능이 좋은 알고리즘 (나무말고 숲을 봄)

- 정형 데이터 : 데이터베이스나 엑셀로 표현할 수 있는 데이터, 앙상블 학습이 효과가 좋음
- 비정형 데이터 : 데이터베이스나 엑셀로 표현하기 어려운 규칙성이 없는 데이터, 신경망 알고리즘을 사용해야함

### 1. 랜덤 포레스트

- 안정적인 성능
- 훈련 데이터에서 랜덤하게 샘플을 추출

In [3]:
# 데이터 준비
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('https://bit.ly/wine_csv_data')
input_data = df[['alcohol','sugar','pH']].to_numpy()
target_data = df['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(input_data, target_data, test_size=0.2, random_state=42)

print('훈련입력데이터:',train_input.shape)
print('테스트입력데이터',test_input.shape)
print('훈련타깃데이터:',train_target.shape)
print('테스트타깃데이터:',test_target.shape)

훈련입력데이터: (5197, 3)
테스트입력데이터 (1300, 3)
훈련타깃데이터: (5197,)
테스트타깃데이터: (1300,)


In [6]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# n_jobs = -1 은 컴퓨터의 모든 CPU를 사용
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs = -1)
# 하이퍼 파라미터 조정 단계는 생략

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [7]:
rf.fit(train_input, train_target)

# 결정트리 특성 중요도 평가 : [0.12345626 0.86862934 0.0079144]
# 랜덤 포레스트 특성 중요도 평가
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [9]:
# oob_score = True 를 하면 자체적으로 모델을 평가하여 점수 산출
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


### 2. 엑스트라 트리

랜덤 포레스트에서 중복허용샘플링을 하지 않고 전체를 대상으로 훈련

과대적합을 줄이고 검증점수를 높여줌

학습 시간이 적음

In [11]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [12]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


### 3. 그레이디언트 부스팅